In [5]:
import numpy as np
import pandas as pd
import spacy
import pdfplumber 
import nltk

jd_df = pd.read_csv('job_title_des.csv')
jd_df.head()

,Unnamed: 0,Job Title,Job Description
0,0,Flutter Developer,We are looking for hire experts flutter develo...
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n..."
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...
4,4,Full Stack Developer,job responsibility full stack engineer – react...


In [6]:
print(jd_df.isnull().sum())
print(jd_df.duplicated().sum())

Unnamed: 0         0
Job Title          0
Job Description    0
dtype: int64
0


In [7]:
jd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2277 entries, 0 to 2276
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       2277 non-null   int64 
 1   Job Title        2277 non-null   object
 2   Job Description  2277 non-null   object
dtypes: int64(1), object(2)
memory usage: 53.5+ KB


In [8]:
import re
def clean_jd(job):
    job = job.lower().strip()
    job = re.sub(r'<.*?>', '', job)
    job = re.sub(r'\s+', ' ', job)
    job = re.sub(r'[^a-zA-Z0-9\s]', '', job)
    return job

jd_df["cleaned_jd"] = jd_df["Job Description"].apply(clean_jd)
print(jd_df[["Job Description","cleaned_jd"]].head())

                                     Job Description  \
0  We are looking for hire experts flutter develo...   
1  PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...   
2  Data Scientist (Contractor)\n\nBangalore, IN\n...   
3  JOB DESCRIPTION:\n\nStrong framework outside o...   
4  job responsibility full stack engineer – react...   

                                          cleaned_jd  
0  we are looking for hire experts flutter develo...  
1  pythondjango developerlead  job codepdj  04 st...  
2  data scientist contractor bangalore in respons...  
3  job description strong framework outside of io...  
4  job responsibility full stack engineer  react ...  


In [10]:
jd_df.head()

,Unnamed: 0,Job Title,Job Description,cleaned_jd
0,0,Flutter Developer,We are looking for hire experts flutter develo...,we are looking for hire experts flutter develo...
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...,pythondjango developerlead job codepdj 04 st...
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n...",data scientist contractor bangalore in respons...
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...,job description strong framework outside of io...
4,4,Full Stack Developer,job responsibility full stack engineer – react...,job responsibility full stack engineer react ...


In [11]:
print(jd_df.iloc[4,3])

job responsibility full stack engineer  react role make impact petsmart transforming engineering team meet need rapidly changing retail environment role foundational helping build craft petsmart  prowess making react j native key framework across mobile web property working across web mobile application part building amazing mobile first customer experience impact customer store digital channel goal responsibility include act full stack developer across petsmart  various web mobile solution full stack engineer help build petsmart  competency using react j native across mobile web property partner web engineering team strategy share component across application best practice new advancement react community work product ux team review design concept offer suggestion decrease complexity maintaining spirit experience proactively make recommendation technical enhancement better performance scalability maintainability accountable delivering agile environment createsoversees completion techni

In [12]:
import re 
#extract job type
def extract_job_type(res):
    job_types = ["full-time", "part-time", "contract", "remote", "hybrid", "internship"]
    for job_type in job_types:
        if job_type in res.lower():
            return job_type.capitalize()
    return "Not Specified"

#extract experience
def extract_exp(resume):
    exp_pattern = r"(\d+)\s*(?:\+|years|yrs|year|yr)\s*(?:experience|exp)"
    isMatch = re.search(exp_pattern,resume)
    if isMatch:
        return (isMatch.group(1))
    
    exp_levels = {
        "entry": 0,
        "fresher": 0,
        "junior": 1,
        "mid": 3,
        "experienced": 5,
        "lead": 4,
        "senior": 7,
        "manager": 10
    }
    for level, years in exp_levels.items():
        if level in resume:
            return years
       
# Apply function to JDs

jd_df["location"] = jd_df["cleaned_jd"].apply(extract_location)
jd_df["job_type"] = jd_df["cleaned_jd"].apply(extract_job_type)
jd_df["experience_required"] = jd_df["cleaned_jd"].apply(extract_exp)

In [13]:
jd_df[["location","job_type","experience_required"]].head()

,location,job_type,experience_required
0,Not Specified,Remote,None
1,Not Specified,Not Specified,4
2,Not Specified,Contract,10
3,Not Specified,Not Specified,None
4,Not Specified,Not Specified,5


In [15]:
#defining skill set
skill_set = {
    # Programming Languages
    "python", "java", "javascript", "c", "c++", "c#", "go", "ruby", "typescript", "php", "swift", "kotlin", "r", "perl",
    
    # Web Development
    "html", "css", "sass", "less", "react", "vue", "angular", "next.js", "nuxt.js", "svelte", "jquery",

    # Backend & APIs
    "node.js", "express", "django", "flask", "spring boot", "fastapi", "laravel", "ruby on rails",
    "rest api", "graphql", "grpc", "microservices", "soap",

    # Databases & Storage
    "mysql", "postgresql", "sqlite", "mongodb", "cassandra", "redis", "firebase", "dynamodb", "elasticsearch",
    "neo4j", "oracle", "sql server", "couchdb", "bigquery",

    # DevOps & Cloud
    "docker", "kubernetes", "aws", "azure", "gcp", "terraform", "ansible", "jenkins", "github actions",
    "kafka", "rabbitmq", "nginx", "apache", "cloud functions", "serverless", "ci/cd", "helm", "istio",

    # Data Science & ML
    "machine learning", "deep learning", "pandas", "numpy", "scikit-learn", "tensorflow", "pytorch",
    "opencv", "huggingface", "nltk", "spacy", "transformers", "matplotlib", "seaborn", "xgboost",

    # Mobile Development
    "android", "ios", "flutter", "react native", "swift", "kotlin", "cordova", "xamarin",

    # Cybersecurity
    "penetration testing", "burpsuite", "wireshark", "nmap", "metasploit", "osint", "siem",
    "owasp", "reverse engineering", "malware analysis",

    # Other Technologies & Tools
    "git", "github", "gitlab", "bitbucket", "jira", "confluence", "agile", "scrum", "trello", "kanban",
    "vim", "bash", "powershell", "zsh", "linux", "windows", "macos"
}
def extract_skills(resume):
    resume = resume.lower()  # Normalize text
    matched_skills = {skill for skill in skill_set if re.search(rf"\b{re.escape(skill)}\b", resume)}
    return list(matched_skills)  # Convert to list for easier processing

# Apply function to cleaned job descriptions
jd_df["extracted_skills"] = jd_df["cleaned_jd"].apply(extract_skills)

In [16]:
jd_df.head()

,Unnamed: 0,Job Title,Job Description,cleaned_jd,location,job_type,experience_required,extracted_skills
0,0,Flutter Developer,We are looking for hire experts flutter develo...,we are looking for hire experts flutter develo...,Not Specified,Remote,None,[flutter]
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...,pythondjango developerlead job codepdj 04 st...,Not Specified,Not Specified,4,"[python, linux]"
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n...",data scientist contractor bangalore in respons...,Not Specified,Contract,10,"[machine learning, pytorch, python, deep learn..."
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...,job description strong framework outside of io...,Not Specified,Not Specified,None,[ios]
4,4,Full Stack Developer,job responsibility full stack engineer – react...,job responsibility full stack engineer react ...,Not Specified,Not Specified,5,"[vue, angular, html, javascript, agile, react]"


In [26]:
import re

indian_cities = ["Mumbai", "Delhi", "Bangalore", "Hyderabad", "Chennai", "Kolkata", "Pune", 
                 "Ahmedabad", "Jaipur", "Surat", "Lucknow", "Kanpur", "Nagpur", "Indore", 
                 "Bhopal", "Patna", "Ludhiana", "Agra", "Nashik", "Vadodara", "Meerut",
                 "Varanasi", "Raipur", "Ranchi", "Guwahati", "Chandigarh", "Coimbatore"]

countries = ["India", "USA", "United States", "UK", "United Kingdom", "Canada", "Australia",
             "Germany", "France", "Singapore", "UAE", "Dubai", "Netherlands", "Japan"]

def extract_location(text):
    if not isinstance(text, str) or text.lower() in ["not specified", "nan"]:
        return "Not Specified"
    
    for city in indian_cities:
        if re.search(rf'\b{city}\b', text, re.IGNORECASE):
            return city + ", India"

    for country in countries:
        if re.search(rf'\b{country}\b', text, re.IGNORECASE):
            return country

    return "Not Specified" 

# Apply function to the location column
jd_df["location"] = jd_df["location"].apply(extract_location)

In [27]:
jd_df["location"].unique()

array(['Not Specified', 'Ahmedabad, India', 'Japan', 'Bangalore, India',
       'India', 'Indore, India', 'Jaipur, India', 'Coimbatore, India',
       'Hyderabad, India', 'Vadodara, India', 'Mumbai, India',
       'Surat, India', 'Chennai, India', 'USA', 'Pune, India',
       'Delhi, India', 'Chandigarh, India', 'Kolkata, India',
       'United States', 'Nashik, India', 'UK', 'Nagpur, India'],
      dtype=object)

In [30]:
jd_df.drop(columns=["extracted_location"])

,Unnamed: 0,Job Title,Job Description,cleaned_jd,location,job_type,experience_required,extracted_skills
0,0,Flutter Developer,We are looking for hire experts flutter develo...,we are looking for hire experts flutter develo...,Not Specified,Remote,None,[flutter]
1,1,Django Developer,PYTHON/DJANGO (Developer/Lead) - Job Code(PDJ ...,pythondjango developerlead job codepdj 04 st...,Not Specified,Not Specified,4,"[python, linux]"
2,2,Machine Learning,"Data Scientist (Contractor)\n\nBangalore, IN\n...",data scientist contractor bangalore in respons...,Not Specified,Contract,10,"[machine learning, pytorch, python, deep learn..."
3,3,iOS Developer,JOB DESCRIPTION:\n\nStrong framework outside o...,job description strong framework outside of io...,Not Specified,Not Specified,None,[ios]
4,4,Full Stack Developer,job responsibility full stack engineer – react...,job responsibility full stack engineer react ...,Not Specified,Not Specified,5,"[vue, angular, html, javascript, agile, react]"
...,...,...,...,...,...,...,...,...
2272,2399,Backend Developer,Job Summary\nPublished on : 26 days ago\nVacan...,job summary published on 26 days ago vacancy ...,Not Specified,Not Specified,02,"[mysql, html, css, postgresql, php]"
2273,2400,Full Stack Developer,business entity cisco umbrella focus cloud-bas...,business entity cisco umbrella focus cloudbase...,Not Specified,Not Specified,4,"[microservices, aws, javascript, jenkins, agil..."
2274,2401,Network Administrator,Urgently reqd in a college in Mohali\nNetwork ...,urgently reqd in a college in mohali network a...,Not Specified,Not Specified,None,[]
2275,2402,Machine Learning,Key Responsibilities: Team leads for small or ...,key responsibilities team leads for small or m...,Not Specified,Not Specified,3,[]


In [32]:
jd_df["location"].value_counts()

location
Not Specified        1933
Bangalore, India       69
Hyderabad, India       40
Mumbai, India          38
Ahmedabad, India       32
Pune, India            27
Chennai, India         26
India                  21
Delhi, India           17
Surat, India           16
Kolkata, India         12
Coimbatore, India      11
Indore, India          10
Jaipur, India           7
Chandigarh, India       4
Vadodara, India         3
Japan                   3
United States           2
Nashik, India           2
Nagpur, India           2
USA                     1
UK                      1
Name: count, dtype: int64

In [33]:
jd_df["job_type"].value_counts()

job_type
Not Specified    1487
Remote            591
Contract          141
Internship         30
Hybrid             28
Name: count, dtype: int64

In [35]:
#removing all JDs where location and job_type not specified
jd_df = jd_df[
    (jd_df["location"].str.lower() != "not specified") & 
    (jd_df["job_type"].str.lower() != "not specified")
]

In [37]:
jd_df["location"].value_counts()

location
Bangalore, India     17
Hyderabad, India     16
Ahmedabad, India     13
Chennai, India       12
Mumbai, India        11
Pune, India          10
India                 7
Indore, India         5
Delhi, India          5
Kolkata, India        4
Surat, India          3
Japan                 3
Chandigarh, India     3
Coimbatore, India     3
Jaipur, India         3
Vadodara, India       2
Nashik, India         1
Nagpur, India         1
United States         1
Name: count, dtype: int64

In [38]:
jd_df.reset_index(drop=True, inplace=True)
jd_df["jd_id"] = jd_df.index 

/var/folders/wr/7srnjrj10kjbvf8f4jp0fvgh0000gn/T/ipykernel_73986/2767661097.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jd_df["jd_id"] = jd_df.index


In [39]:
jd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           120 non-null    int64 
 1   Job Title            120 non-null    object
 2   Job Description      120 non-null    object
 3   cleaned_jd           120 non-null    object
 4   location             120 non-null    object
 5   job_type             120 non-null    object
 6   experience_required  49 non-null     object
 7   extracted_skills     120 non-null    object
 8   extracted_location   120 non-null    object
 9   jd_id                120 non-null    int64 
dtypes: int64(2), object(8)
memory usage: 9.5+ KB


In [41]:
jd_df.drop(columns=["Unnamed: 0"])

,Job Title,Job Description,cleaned_jd,location,job_type,experience_required,extracted_skills,extracted_location,jd_id
0,Wordpress Developer,Position: WordPress Developer\nLocation: Prahl...,position wordpress developer location prahladn...,"Ahmedabad, India",Remote,None,"[jquery, javascript, git]","Ahmedabad, India",0
1,JavaScript Developer,"No relocation needed, can telework from home, ...",no relocation needed can telework from home re...,Japan,Remote,10,"[express, aws, javascript, postgresql, jquery]",Japan,1
2,Django Developer,Selected intern's day-to-day responsibilities ...,selected interns daytoday responsibilities inc...,"Indore, India",Internship,None,"[mysql, python, c, django]","Indore, India",2
3,PHP Developer,"Teamwork php back end development, maintenance...",teamwork php back end development maintenance ...,"Vadodara, India",Remote,None,"[mysql, php, git]","Vadodara, India",3
4,iOS Developer,We have urgent requirement at Netra Technosys....,we have urgent requirement at netra technosys ...,"Surat, India",Remote,None,"[ios, swift]","Surat, India",4
...,...,...,...,...,...,...,...,...,...
115,JavaScript Developer,Bonami Software is looking for enthusiastic *J...,bonami software is looking for enthusiastic ja...,"Delhi, India",Remote,4,"[vue, angular, express, aws, javascript, react...","Delhi, India",115
116,Backend Developer,We are looking for a backend developer for Aur...,we are looking for a backend developer for aur...,"Coimbatore, India",Remote,3,[],"Coimbatore, India",116
117,Flutter Developer,· Design and build advanced applications for t...,design and build advanced applications for th...,"Vadodara, India",Remote,None,[flutter],"Vadodara, India",117
118,Network Administrator,This specific role is focused on expertise in ...,this specific role is focused on expertise in ...,"Jaipur, India",Remote,None,"[gcp, azure, aws]","Jaipur, India",118


In [42]:
jd_df.to_csv("jd.csv", index=False)  